# PyMem3DG Tutorial 3 - Open Boundary Simulation and Masking
`Cuncheng Zhu, Christopher T. Lee`

This tutorial covers the functionality of PyMem3DG on dealing with boundary condition. In previous tutorials, we only simulate closed membrane and therefore no addditional complexity on the boundary. Though in reality, lipid membrane is almost always closed, it is beneficial to only simulate certain region of the membrane to save computational cost. 

To demonstrate, we will set up a single boundary system on a hexagon patch and conduct study on the effect of osmotic pressure similar to previous tutorial, but instead of simulating the full vesicle, we zoom in at a small region on the surface. 

In [ ]:
import pymem3dg as dg
import pymem3dg.util as dg_util
import pymem3dg.boilerplate as dgb
from functools import partial
from pathlib import Path

There is a pregenerated patch mesh in `/sample_meshes` folder that has circular boundary, but we could also use a built-in function to generate a hexgon mesh and subdivide them to get enough resolution, like this:

<img src="output/tutorial3/screenshot_frame000000.png" width="400" height="200">


In [ ]:
hexFace, hexVertex = dg.getHexagon(radius = 1, subdivision = 4)
geo = dg.Geometry(hexFace, hexVertex)

p = dg.Parameters()
p.bending.Kbc = 8.22e-5 
p.tension.form = partial(
    dgb.preferredAreaSurfaceTensionModel, modulus=0.1, preferredArea=103, reservoirArea = 100
)
p.osmotic.form = partial(
    dgb.preferredVolumeOsmoticPressureModel,
    preferredVolume=5,
    reservoirVolume=4,
    strength=0.1,
)

This is the same setup as in tutorial 2. However, we will have addition parameter to provide information at and outside the boundary. Again, we are simulating *part* of close membrane structure! 

Outside of boundary, $V_{res}$ and $A_{res}$ represents the remaining volume and surface area reservoir such that the reference volume $\bar{V} = V_{res}$ and $\bar{A} = A_{res} + A_{polygon}$, where $A_{polygon}$ is total polygon area formed by the boundary loop polygon.

At the boundary, we need to specify the boundary condition, the option includes $\texttt{"none"}$, $\texttt{"fixed"}$, $\texttt{"pin"}$ and $\texttt{"roller"}$. We will demonstrate how it affects the simulations in the following examples and please refer to the documentation for specfics and implementation if necessary.

## Fixed boundary 
The fixed boundary condition fix both the position of the boundary elements, but also their curvature.

In [ ]:
p.boundary.shapeBoundaryCondition = "fixed"

g = dg.System(geometry=geo, parameters=p)
g.initialize()

outputDir = Path("output/tutorial3/fixed")
outputDir.mkdir(parents=True, exist_ok=True)

fe = dg.Euler(g, 1, 10000, 1000, 3e-10, str(outputDir))
fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True
fe.integrate()

In [ ]:
import pymem3dg.visual as dg_vis
import polyscope as ps
# dg_vis.animate(str(outputDir / "traj.nc"), meanCurvature = True)
# ps.show()

At the final time $T = 10000$, resisting the surface tension, the osmotic pressure push the membrane outward, forming some bump.

## Pin boundary
The pin boundary condition only fix the position of the boundary, but not the curvature.

In [ ]:
p.boundary.shapeBoundaryCondition = "pin"

g = dg.System(geometry=geo, parameters=p)
g.initialize()

outputDir = Path("output/tutorial3/pin")
outputDir.mkdir(parents=True, exist_ok=True)

fe = dg.Euler(g, 1, 10000, 1000, 3e-10, str(outputDir))

fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True
success = fe.integrate()

At the same final time $T = 10000$, the boundary is no longer flat.

## Roller boundary

In [ ]:
p.boundary.shapeBoundaryCondition = "roller"

g = dg.System(geometry=geo, parameters=p)
g.meshProcessor.meshMutator.collapseSkinny = True
g.meshProcessor.meshMutator.collapseSmall = True
g.initialize()

outputDir = Path("output/tutorial3/roller")
outputDir.mkdir(parents=True, exist_ok=True)

fe = dg.Euler(g, 1, 10000, 1000, 3e-10, str(outputDir))

fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True
fe.integrate()

In this case, we get shape with shrunken rounded corners.

The roller boundary condition constrains movements in certain direction, at this case in the vertical direction but allowing sliding over the horizontal plane. 

You might notice that we add edge collapse to this set of simulation. This is due to the fact where rounded boundary can create skinny triangles at the corner that should be eliminated. 

## Mask
Under the hood, the boundary condition is implemented using mask. In addition to boundary mask, we could also mask vertices based on particular neighborhood, which applies to both open and closed mesh. The specifics about how to specify these vertices in PyMem3DG will be covered in anthother tutorial, but just for completeness and demonstration, we could have some area remaining flat by doing 

In [ ]:
p.boundary.shapeBoundaryCondition = "fixed"
p.point.pt = [1,1]
p.variation.geodesicMask = 1.3

g = dg.System(geometry=geo, parameters=p)
g.initialize()

outputDir = Path("output/tutorial3/mask")
outputDir.mkdir(parents=True, exist_ok=True)

fe = dg.Euler(g, 1, 10000, 1000, 3e-10, str(outputDir))

fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True
fe.integrate()

In this case, the result the mesh will only budge in a small section around `pt` in radius of 1.3.

## Additional Notes:
### $A_{polygon}$
**Please note that in current version of PyMem3DG, $A_{polygon}$ is only well-defined when the boundary loop remains on the same plane!** If not, the area value will depend on how the underlying algorithm divide the boundary polygon, which is not fully predictable. 
### Reference mesh
The reasoning on specifying reference mesh remain the same on open boundary mesh vs close surface mesh. In most cases, self referencing is sufficient. One notable difference between close mesh and open mesh is the reference area, which will remain as long as boundary position is fixed (a.k.a fixed and pin boundary). Additional reference mesh may be needed when doing roller boundary condition when for example, continuing an interrupted simulation. 


### Constant osmotic pressure & surface tension 
Instead of following the argument of having a membrane reservoir, maybe it is more explicit to specify the surface tension and pressure directly! This is particularly true when membrane reservoir is so large that local deformation is too unnoticeable that approximately it is just applying some constant value of pressure and tension. To do this explicitly, you can replace the functional forms of the tension and osmotic pressure calculation. Helper functions are provided in the boilerplates.

In [ ]:
p.tension.form = partial(
    dgb.constantSurfaceTensionModel, tension = 1e-4)

p.osmotic.form = partial(
    dgb.constantOsmoticPressureModel, pressure = 1e-3
)